# Storing Storm Data

### Introduction

In this project, I will the storm_data from [here](https://dq-content.s3.amazonaws.com/251/storm_data.csv), a csv that simulates hurricane data. The csv contains the following fields:

- fid - ID for the row
- year - Recorded year
- month - Recorded month
- day - Recorded date
- ad_time - Recorded time in UTC
- btid - Hurricane ID
- name - Name of the hurricane
- lat - Latitude of the recorded location
- long - Longitude of the recorded location
- wind_kts - Wind speed in knots per second
- pressure - Atmospheric pressure of the hurricane
- cat - Hurricane category
- basin - The basin the hurricane is located
- shape_leng - Hurricane shape length

The objective of the project is to take this data and add into a database. To create the database, we will use a local version of PostgreSQL.

### Importing the data

In [48]:
import io
from urllib import request
import csv
import psycopg2

response = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
reader = csv.reader(io.TextIOWrapper(response))

line_number = 0
for line in reader:
    if line_number < 5:
        print(line)
        line_number +=1
    else:
        break

['ï»¿FID', 'YEAR', 'MONTH', 'DAY', 'AD_TIME', 'BTID', 'NAME', 'LAT', 'LONG', 'WIND_KTS', 'PRESSURE', 'CAT', 'BASIN', 'Shape_Leng']
['2001', '1957', '8', '8', '1800Z', '63', 'NOTNAMED', '22.5', '-140', '50', '0', 'TS', 'Eastern Pacific', '1.140175']
['2002', '1961', '10', '3', '1200Z', '116', 'PAULINE', '22.1', '-140.2', '45', '0', 'TS', 'Eastern Pacific', '1.16619']
['2003', '1962', '8', '29', '0600Z', '124', 'C', '18', '-140', '45', '0', 'TS', 'Eastern Pacific', '2.10238']
['2004', '1967', '7', '14', '0600Z', '168', 'DENISE', '16.6', '-139.5', '45', '0', 'TS', 'Eastern Pacific', '2.12132']


### Creating the table

In [49]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="upsdflintmobile")
cur = conn.cursor()

query1 = "DROP TABLE IF EXISTS storm"
query2 ="""CREATE TABLE storm(
id INTEGER PRIMARY KEY,
date TIMESTAMP,
btid INTEGER,
name VARCHAR(15),
latitude DECIMAL(4,1),
longitude DECIMAL(4,1),
wind_kts INTEGER,
pressure INTEGER,
cat VARCHAR(3),
basin VARCHAR(16),
shape_leng DECIMAL(8,6)
);
"""

cur.execute(query1)
cur.execute(query2)
conn.commit()
conn.close()

### Creating users and giving permissions

I will create two users: a data_production, that can insert, update and read the data but not delete it and a data_analyst, that can only run queries to read the data.

In [52]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="upsdflintmobile")
cur = conn.cursor()

cur.execute("""CREATE USER data_production WITH NOSUPERUSER PASSWORD 'production'""")
cur.execute('REVOKE ALL ON storm FROM data_production')
cur.execute('GRANT SELECT, INSERT, UPDATE ON storm TO data_production')

cur.execute("""CREATE USER data_analyst WITH NOSUPERUSER PASSWORD 'analyst'""")
cur.execute('REVOKE ALL ON storm FROM data_analyst')
cur.execute('GRANT SELECT ON storm TO data_analyst')

conn.commit()
conn.close()

### Inserting the data

First we read the csv into Pandas, so we can transform the YEAR, MONTH, DAY and AD_TIME columns into a timestamp. Then we export it into a csv and import the data into the database.

Finally, we select the first line of the database as an example.

In [53]:
import pandas as pd

df = pd.read_csv('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
df['HOUR'] = df['AD_TIME'].map(lambda x: str(x)[0:2])
df['MINUTE'] = df['AD_TIME'].map(lambda x: str(x)[2:4])
df['DATE'] = pd.to_datetime(df[['DAY','MONTH','YEAR', 'HOUR', 'MINUTE']]
                   .astype(str).apply(' '.join, 1), format='%d %m %Y %H %M')
df.drop(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'AD_TIME'], axis=1, inplace=True)
df = df[['FID', 'DATE', 'BTID', 'NAME', 'LAT', 'LONG', 'WIND_KTS', 'PRESSURE', 'CAT','BASIN', 'Shape_Leng']]
df.to_csv('processed_data.csv', index=False)

In [54]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="upsdflintmobile")
cur = conn.cursor()

with open('processed_data.csv', 'r') as f:
    cur.copy_expert('COPY storm FROM STDIN WITH CSV HEADER', f)

    conn.commit()
conn.close()

In [55]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="upsdflintmobile")
cur = conn.cursor()
cur.execute('SELECT * FROM storm')
results = cur.fetchone()
conn.close()

results

(2001,
 datetime.datetime(1957, 8, 8, 18, 0),
 63,
 'NOTNAMED',
 Decimal('22.5'),
 Decimal('-140.0'),
 50,
 0,
 'TS',
 'Eastern Pacific',
 Decimal('1.140175'))